## Pipeline de disponibilização, catalogação e preprocessamento de relatorios

### Funções de captura e disponibilização de relatórios

In [10]:
# Bibliotecas

import pandas as pd
import io
import requests
import zipfile
import pandas as pd
import re
from pathlib import Path
import os

root = os.path.dirname("C:\\Users\\thgcn\\OneDrive\\Academico\\Mestrado - NLP - Finance\\datasets\\")
#levelone = ['10456', '24600', '906', '1210', '1325', '21199', '18724', '4820', '2437', '2453', '3069', '3077', '18376', '16632', '5770', '6211', '3980', '8672', '19348', '7617', '8656', '11312', '14109', '14320']
levelone = ['19348']

In [3]:
# Função para download da relação de todos os links
def busca_ipe(ano):
    url = ('https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/IPE/DADOS/ipe_cia_aberta_%d.zip' %(ano))
    file = ('ipe_cia_aberta_%d.zip' %(ano))    
    r = requests.get(url)
    zf = zipfile.ZipFile(io.BytesIO(r.content))   
    file = zf.namelist()  
    zf = zf.open(file[0])
    lines = zf.readlines()
    lines=[i.strip().decode('ISO-8859-1') for i in lines] 
    lines=[i.split(';') for i in lines]
    qtl = len(lines)     
    return lines

# Função para busca de um item específico
def search (lista, valor):
    return [(lista[lista.index(x)]) for x in lista if valor in x]

# Função para baixar o arquivivo específico
def baixar_arquivo(url, endereco):
    resposta = requests.get(url, allow_redirects=True, verify=False, stream=True)
    if resposta.status_code == requests.codes.OK:
        with open(endereco, 'wb') as novo_arquivo:
                novo_arquivo.write(resposta.content)
        print("Download finalizado. Arquivo salvo em: {}".format(endereco))
    else:
        resposta.raise_for_status()
        
#Funcão para baixar arquivos específicos
def download_def(year):
    lines = busca_ipe(year)
    defi = search(lines,"Dados Econômico-Financeiros")
    year = year
    for a in range(len(defi)):
       if defi[a][2] in levelone:
            sufix = re.sub(u'[^a-zA-Z0-9]','_',defi[a][2]+"_"+defi[a][1])
            category = re.sub(u'[^a-zA-Z0-9çãàáêéíõóú]','_',defi[a][5])
            print(sufix)
            company = os.path.join(root + "\\%s"%(sufix))
            print(company)
       if not Path(root + "\\%s"%(sufix)).is_dir():
            print("nao existe")
            os.makedirs(root + "\\%s"%(sufix))
       if not Path(company +"\\%d"%(year)).is_dir():
            print("nao existe")
            os.makedirs(company + "\\%d"%(year))
       diryear = os.path.join(company + "\\%d"%(year))
       if not Path(diryear +"\\%s"%(category)).is_dir():
            print("nao existe")
            os.makedirs(diryear + "\\%s"%(category))
       dircategory = os.path.join(diryear + "\\%s"%(category))
       url = defi[a][12]       
       nome = re.sub(u'[^a-zA-Z0-9]', '_', defi[a][2]+"_"+defi[a][1]+"_"+defi[a][7][1:50]+"_"+defi[a][8])[1:100]       
       baixar_arquivo(url, dircategory+"\\%s.pdf"%(nome))

### Funções de preprocessamento de arquivos

In [7]:
# Bibliotecas e modelos


from transformers import LongformerModel, LongformerTokenizer

# Carregar o tokenizer e o modelo pré-treinado
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')

from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForPreTraining, BertForQuestionAnswering, AutoModel, AutoTokenizer, BertTokenizer, BertForQuestionAnswering, BertForSequenceClassification
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
import PyPDF2
import re
import numpy as np

import spacy
# Carrrefa os modelos
#model = AutoModel.from_pretrained('neuralmind/bert-large-portuguese-cased')
#model = BertForSequenceClassification.from_pretrained('neuralmind/bert-large-portuguese-cased')
model_forquestion = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
#tokenizer = AutoTokenizer.from_pretrained('neuralmind/bert-large-portuguese-cased')
#model = AutoModelForPreTraining.from_pretrained("neuralmind/bert-large-portuguese-cased")

# Instanciar o modelo e o tokenizador T5 pré-treinado
modelsum = T5ForConditionalGeneration.from_pretrained('t5-base')
tokenizersum = T5Tokenizer.from_pretrained('t5-base')


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:

def convert_pdf (doc):
    with open(doc, "rb") as f:
        # Use a biblioteca PyPDF2 para ler o arquivo
        #reader = PyPDF2.PdfFileReader(f)
        reader = PyPDF2.PdfReader(f)
        # Obtenha o número de páginas no arquivo
        #num_pages = reader.getNumPages()
        num_pages = len(reader.pages)
        # Crie uma string vazia para armazenar o texto extraído
        text = ''
        # Itere pelas páginas e extraia o texto
        for page in range(num_pages):
            # Obtenha o objeto da página
            #page_obj = reader.getPage(page)
            page_obj = reader.pages[page]
            # Extraia o texto da página
            #page_text = page_obj.extractText() 
            page_text = page_obj.extract_text()
            # Adicione o texto extraído à string de texto
            text += page_text
    return text
            

def remover_stopwords(tokens):
    # Carrega o modelo do SpaCy para o idioma português
    nlp = spacy.load('pt_core_news_lg')

    # Cria uma lista para armazenar os tokens sem stopwords
    tokens_sem_stopwords = []

    # Percorre cada token na lista
    for token in tokens:
        # Verifica se o token não é uma stopword
        if not nlp.vocab[token].is_stop:
            tokens_sem_stopwords.append(token)
    return tokens_sem_stopwords


def normalize_text(text):
    text=text
    # Converte o texto para minúsculas
    normalized_text = text.lower()
    # Remove caracteres especiais e acentuação
    normalized_text = re.sub(r'[^\w\s]', '', normalized_text)
    # Retorna o texto limpo
    return normalized_text

def tokengen(text):
    tokens = tokenizer.encode(text, add_special_tokens=True) 
    return tokens

def vector___one(text):
    max_length = 512
    input_parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    # Lista para armazenar os vetores médios
    vectors = []
    # Processa cada parte do texto
    for part in input_parts:
    # Tokeniza a parte do texto
        tokens = tokenizer.encode(part, add_special_tokens=True)
        input_ids = torch.tensor([tokens])
        # Gera os vetores de embeddings
        with torch.no_grad():
            outputs = model(input_ids)
            last_hidden_states = outputs[0]
        # Calcula o vetor médio
        average_vector = torch.mean(last_hidden_states, dim=1)
        # Adiciona o vetor médio à lista
        vectors.append(average_vector)
    # Concatena os vetores gerados
    text_vector = torch.cat(vectors, dim=1)
    return text_vector


def vector_one(text):
    tokens = tokenizer.encode(text, add_special_tokens=True)
    max_length = 4096
   
    # Divide os tokens em partes de tamanho 4096
    token_parts = [tokens[i:i+max_length] for i in range(0, len(tokens), max_length)]
    
    # Converte as partes de tokens em tensores
    tensor_parts = [torch.tensor(part) for part in token_parts]
    
    # Concatena os tensores das partes em um único tensor
    concatenated_tensor = torch.cat(tensor_parts)
    concatenated_tensor.shape
    print(len(tokens))
    return concatenated_tensor
    

def vector____one(text):
    max_length = 4096
    average_vector = []
    vectors = []
    input_parts = []
    
    # Tokenize the text
    tokens = tokenizer.encode(text, add_special_tokens=True, truncation=True, max_length=4096, padding='longest', return_tensors='pt')
    input_ids = tokens.clone().detach()
    
    # Divide the tokenized text into parts
    input_parts = [tokens[0, i:i+max_length] for i in range(0, tokens.shape[1], max_length)]
    
    # Process each part of the tokenized text
    for part in input_parts:
        # Generate the embeddings vectors
        with torch.no_grad():
            outputs = model(input_ids[:, part])
            last_hidden_states = outputs[0]
        
        # Calculate the average vector
        average_vector = torch.mean(last_hidden_states, dim=1)
        
        # Add the average vector to the list
        vectors.append(average_vector)
    
    # Concatenate the generated vectors
    text_vector = torch.cat(vectors, dim=1)
    return text_vector 


def similarity_vector(a, b):
    size_a = a.size(0)
    size_b = b.size(0)
    if size_a == size_b:
        #similarity_score = cosine_similarity(a, b)
        similarity_score = cosine_similarity(a.float(), b.float())
        return similarity_score.item()
    if size_a > size_b:
        diff = size_a - size_b
        padded = F.pad(b, pad=(0, diff), mode='constant', value=0)
        similarity_score = cosine_similarity(a.float(), padded.float())
        return similarity_score.item()
    if size_a < size_b:
        diff = size_b - size_a
        padded = F.pad(a, pad=(0, diff), mode='constant', value=0)
        similarity_score = cosine_similarity(b.float(), padded.float())
        return similarity_score.item()
    

    
def question(text, question):
    question = question
    max_answer_length = 512
    max_length = 512
    input_parts = []
    input_parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]
    # Lista para armazenar as respostas
    answers = []
    # Processa cada parte do texto
    for part in input_parts:
        # Tokeniza a parte do texto
        # Tokeniza a pergunta e o texto menor
        encoding = tokenizer.encode_plus(question, part, return_tensors='pt', max_length=512, truncation=True)
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        # Obtém as respostas do modelo pré-treinado
        with torch.no_grad():
            outputs = model_forquestion(input_ids=input_ids, attention_mask=attention_mask)
            start_scores = outputs.start_logits
            end_scores = outputs.end_logits
        # Obtém a resposta com maior probabilidade
        answer_start = torch.argmax(start_scores)
        answer_end = torch.argmax(end_scores)
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[0][answer_start.item():answer_end.item()+1]))
        # Adiciona a resposta à lista
        answers.append(answer)
    # Combina as respostas
    combined_answer = ' '.join(answers)
    # Limita o tamanho da resposta final
    combined_answer = combined_answer[:max_answer_length]
        # Formatação da resposta final
    formatted_answer = "RESPOSTA:\n\n"
    formatted_answer += combined_answer.replace(".", ".\n\n")
    # Apresenta a resposta final
    return formatted_answer
    # Apresenta a resposta final
    return combined_answer


def summarization(text):
    num_sentences=5
    #max_length=512
    # Tokenizar o texto em partes
    input_parts = text.split('. ')
    summaries = []
    #input_parts = []
    #input_parts = [text[i:i+max_length] for i in range(0, len(text), max_length)]    
    for part in input_parts:
        lenght=len(part)
        m_length = int(lenght*0.4)
        # Tokenizar a parte do texto
        inputs = tokenizersum.encode("summarize: " + part, return_tensors="pt")
        # Gerar o resumo da parte do texto usando o modelo T5
        outputs = modelsum.generate(inputs, max_length=m_length, num_beams=4, early_stopping=True)
        summary = tokenizersum.decode(outputs[0], skip_special_tokens=True)

        # Adicionar o resumo à lista de sumários
        summaries.append(summary)

    # Concatenar os sumários em um único texto
    concatenated_summary = ' '.join(summaries)

    # Extrair as N sentenças mais importantes
    sentences = concatenated_summary.split('. ')
    top_sentences = sorted(sentences, key=lambda x: len(x), reverse=True)[:num_sentences]

    return '. '.join(top_sentences)


def prep_proc(text1,text2):
    nm1=normalize_text(text1)
    nm2=normalize_text(text2)
    print("Texto normalizado 1: " + nm1[:200])
    print("Texto normalizado 1: " + nm2[:200])
    tk1 = tokengen(nm1)
    tk2 = tokengen(nm2)
    vt1=vector_one(nm1)
    vt2=vector_one(nm2)
    print("Vetor do artigo 1:")
    print(vt1.shape)
    print(v1)
    print("Vetor do artigo 2:")
    print(vt2.shape)
    print(v2)
    print("calculo de similaridades entre texto 1 e texto2 (cosseno)")
    sim=similarity_vector(v1,v2)
    print(sim)
    return v1,v2,sim


## Serviço de catalogação, funções de indexação e busca

In [21]:
## Inicialização do serviço de catalogação modo cloud

import subprocess
import os
import requests

# Comando que você deseja executar
command = 'C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\solr restart -port 8983'
#Inicialização em modo cloud
command2 = 'C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\solr -c -z localhost:9983 -p 8984'

# Executa o comando no terminal
subprocess.run(command, shell=True)
#Inicialização em modo cloud
subprocess.run(command2, shell=True)

CompletedProcess(args='C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\solr -c -z localhost:9983 -p 8984', returncode=1)

### Criação da coleção (agrupamento):

In [ ]:
# Caminho para o diretório bin do Solr
solr_bin_path = 'C:\\Users\\thgcn\\OneDrive\\Academico\\PO-245\\Projeto\\solr-9.2.1\\bin\\'

# Comando para criar a coleção no modo standalone
command = 'solr create_collection -c relatoriosfinanceiros -s 2 -rf 2'

os.chdir(solr_bin_path)
os.system(command)

# URL do endpoint Solr
url = 'http://localhost:8983/api/cores/relatoriosfinanceiros/schema'

# Cabeçalhos da solicitação POST
headers = {
    'Content-Type': 'application/json'
}

# Corpo dos campos da catalogação
data = {
    "add-field": [
        {"name": "report_id", "type": "string"},
        {"name": "title", "type": "text_general", "multiValued": False},
        {"name": "company_name", "type": "text_general", "multiValued": False},
        {"name": "cod_cvm", "type": "text_general", "multiValued": False},
        {"name": "year", "type": "pint"},
        {"name": "date", "type": "pdate"},
        {"name": "content", "type": "text_general", "multiValued": True},
        {"name": "keywords", "type": "text_general", "multiValued": True},
        {"name": "size", "type": "text_general", "multiValued": True},
        {"name": "tensor", "type": "text_general","multiValued": True}
    ]
}

# Enviar solicitação POST para atualizar o esquema no Solr
response = requests.post(url, json=data, headers=headers)
if response.status_code == 200:
    print("Esquema atualizado com sucesso.")
else:
    print("Falha ao atualizar o esquema. Status:", response.status_code)

In [15]:
def update_schema(report_id, title, company_name, year, date, content, keywords, size, tensor):
    # URL do endpoint Solr
    url = 'http://localhost:8983/api/cores/relatoriosfinanceiros/schema'

    # Cabeçalhos da solicitação POST
    headers = {
        'Content-Type': 'application/json'
    }

    # Corpo da solicitação POST
    data = {
        "add-field": [
            {"name": "report_id", "type": "string", "value": report_id},
            {"name": "title", "type": "text_general", "multiValued": False, "value": title},
            {"name": "company_name", "type": "text_general", "multiValued": False, "value": company_name},
            {"name": "year", "type": "pint", "value": year},
            {"name": "date", "type": "pdate", "value": date},
            {"name": "content", "type": "text_general", "multiValued": True, "value": content},
            {"name": "keywords", "type": "text_general", "multiValued": True, "value": keywords},
            {"name": "size", "type": "text_general", "multiValued": True, "value": size},
            {"name": "tensor", "type": "text_general", "multiValued": True, "value": tensor}
        ]
    }

    # Enviar solicitação POST para atualizar o esquema no Solr
    response = requests.post(url, json=data, headers=headers)

    if response.status_code == 200:
        print("Esquema atualizado com sucesso.")
    else:
        print("Falha ao atualizar o esquema. Status:", response.status_code)

# Exemplo de uso da função
update_schema("123456", "Relatório de Vendas", "Itau", 2023, "2023-05-15", "Relatorio anual de 2022", "relatorio financeiro itau finanças", "104857600", "12751832")


Falha ao atualizar o esquema. Status: 400


In [16]:
import os
import json

def update_schema(report_id, title, company_name, year, date, content, keywords, size, tensor):
    # URL do endpoint Solr
    url = 'http://localhost:8983/api/cores/relatoriosfinanceiros/schema'

    # Cabeçalhos da solicitação POST
    headers = {
        'Content-Type': 'application/json'
    }

    # Corpo da solicitação POST
    data = {
        "add-field": [
            {"name": "report_id", "type": "string", "value": report_id},
            {"name": "title", "type": "text_general", "multiValued": False, "value": title},
            {"name": "company_name", "type": "text_general", "multiValued": False, "value": company_name},
            {"name": "year", "type": "pint", "value": year},
            {"name": "date", "type": "pdate", "value": date},
            {"name": "content", "type": "text_general", "multiValued": True, "value": content},
            {"name": "keywords", "type": "text_general", "multiValued": True, "value": keywords},
            {"name": "size", "type": "text_general", "multiValued": True, "value": size},
            {"name": "tensor", "type": "text_general", "multiValued": True, "value": tensor}
        ]
    }

    # Serializar o corpo da solicitação POST como uma string JSON
    data_json = json.dumps(data)

    # Comando curl para enviar a solicitação POST
    command = f"curl --request POST --url '{url}' --header 'Content-Type: application/json' --data '{data_json}'"

    # Executar o comando usando os.system()
    os.system(command)

# Exemplo de uso da função
update_schema("123456", "Relatório de Vendas", "Itau", 2023, "2023-05-15", "Relatorio anual de 2022", "relatorio financeiro itau finanças", "104857600", "12751832")


In [ ]:
import os
import json

def update_schema(report_id, title, company_name, year, date, content, keywords, size, tensor):
    # URL do endpoint Solr
    url = 'http://localhost:8983/api/cores/relatoriosfinanceiros/schema'

    # Cabeçalhos da solicitação POST
    headers = {
        'Content-Type': 'application/json'
    }

    # Corpo da solicitação POST
    data = {
        "add-field": [
            {"name": "report_id", "type": "string", "value": report_id},
            {"name": "title", "type": "text_general", "multiValued": False, "value": title},
            {"name": "company_name", "type": "text_general", "multiValued": False, "value": company_name},
            {"name": "year", "type": "pint", "value": year},
            {"name": "date", "type": "pdate", "value": date},
            {"name": "content", "type": "text_general", "multiValued": True, "value": content},
            {"name": "keywords", "type": "text_general", "multiValued": True, "value": keywords},
            {"name": "size", "type": "text_general", "multiValued": True, "value": size},
            {"name": "tensor", "type": "text_general", "multiValued": True, "value": tensor}
        ]
    }

    # Serializar o corpo da solicitação POST como uma string JSON
    data_json = json.dumps(data)

    # Comando curl para enviar a solicitação POST
    command = f"curl --request POST --url '{url}' --header 'Content-Type: application/json' --data '{data_json}'"

    # Exibir o comando curl para depuração
    print("Comando curl:", command)

    # Executar o comando usando os.system()
    return_code = os.system(command)

    # Verificar o código de retorno
    if return_code == 0:
        print("Esquema atualizado com sucesso.")
        return True
    else:
        print("Falha ao atualizar o esquema.")
        return False

# Exemplo de uso da função
update_successful = update_schema("123456", "Relatório de Vendas", "Itau", 2023, "2023-05-15", "Relatorio anual de 2022", "relatorio financeiro itau finanças", "104857600", "12751832")

if update_successful:
    # A solicitação foi bem-sucedida
    print("Solicitação executada com sucesso.")
else:
    # A solicitação falhou
    print("Falha na execução da solicitação.")


## Função principal

In [22]:
def pipeline(empresa, ano):
    doc = download_def(empresa, ano)
    nm=normalize_text(doc)
    print("Texto normalizado: " + nm[:200])
    print("Tokens texto:")
    tk=tokengen(nm)
    vt=vector_one(nm)
    print("Vetor do artigo:")
    print(vt.shape)
    print(vt)
    report_id = doc[0]
    title = doc[1]
    date = doc[2]
    type =doc[3]
    tags = doc[4]
    size =doc[5]    
    update_schema(report_id, title, empresa, ano, date, type, tags, size, vt)

    return doc, report_id, title, date, type, tags, size, vt


## Chamada da função

pipeline(itau, 2022)